---
#### final data 로 ml 학습 


In [13]:
#### Jupyter Basic Setting ####
import pandas as pd,numpy as np
import matplotlib.pyplot as plt,seaborn as sns  # 시각화
import warnings; warnings.filterwarnings('ignore')  # 경고 무시
import sys,os
from pprint import pprint 
parent_dir = os.path.dirname(os.getcwd())
parent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
from Functions import Service as S
# 기본 세팅
S.plotSetting("seaborn-v0_8")
def blue(str):return S.colored_text(str,'blue')
def yellow(str):return S.colored_text(str,'yellow')
def red(str):return S.colored_text(str,'red')
def green(str):return S.colored_text(str,'green')


___ 🟡 PLOTSETTING. 
______ 📌 matplotlibn plot 한글화 Setting
___## OS platform 한글 세팅완료 ## ___


In [8]:
import pandas as pd 
data =pd.read_csv('./Data/final.csv')

In [9]:
data.columns

Index(['Unnamed: 0', '시설아이디', 'weeks', '조사일자', '표본번호', '관부직경', '엽병장', '엽수',
       '엽장', '엽폭', '개화수', '초장', '최종화방차수', '화방 꽃수(소화수)', 'supplyEC', 'supplyPH',
       'innerCO2', 'innerHum', 'innerTemp', 'innerSolar', 'CO2'],
      dtype='object')

In [10]:
input_cols =['weeks','supplyEC', 'supplyPH',
       'innerCO2', 'innerHum', 'innerTemp', 'innerSolar', 'CO2']
output_cols = ['관부직경', '엽병장', '엽수',
       '엽장', '엽폭', '개화수', '초장', '최종화방차수', '화방 꽃수(소화수)']
training_table = data.loc[:,input_cols]
target_table= data.loc[:,output_cols]

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from sklearn.model_selection import cross_val_score


# 예시 데이터 (training_table과 target_table이 이미 존재한다고 가정)
# training_table = pd.DataFrame(...)
# target_table = pd.DataFrame(...)

# 데이터 분할
train_input, test_input, train_target, test_target = \
    train_test_split(training_table,
                     target_table, 
                     test_size=0.2,
                     random_state=42)

def linear_regressor_prdict():
    ## Linear Regression model 비교
    lin_regressor = LinearRegression()
    multi_output_regressor_lin = MultiOutputRegressor(lin_regressor)
    multi_output_regressor_lin.fit(train_input, train_target)
    y_pred_lin = multi_output_regressor_lin.predict(test_input)
    mse = mean_squared_error(test_target, y_pred_lin)
    rmse = np.sqrt(mse)
    
    #### 교차검증 
    scores_cv = cross_val_score(multi_output_regressor_lin,train_input,train_target,scoring='neg_mean_squared_error',cv=10)
    rmse_cv = np.sqrt(-scores_cv)
    print(yellow(f"Linear regression model RMSE: {rmse:.2f}"))
    print("\t ",red(f"LR cv score : {rmse_cv}"))
    print("\t ",green(f"LR cv RMSE  average : {rmse_cv.mean():.2f}"))
linear_regressor_prdict()
def knn_regressor_predict():
    ## KNN regression model
    knn_regressor = KNeighborsRegressor(n_neighbors=3)
    ## Multi Output Setting
    multi_output_regressor_knn = MultiOutputRegressor(knn_regressor)
    multi_output_regressor_knn.fit(train_input, train_target)

    score = multi_output_regressor_knn.score(test_input, test_target)
    y_pred_knn = multi_output_regressor_knn.predict(test_input)
    mse = mean_squared_error(test_target, y_pred_knn)
    rmse = np.sqrt(mse)
    # print(yellow(f'KNN(3) regression model score: {score}'))
    print(yellow(f'KNN(3) regression model RMSE: {rmse:.2f}'))
    #### 교차검증 
    scores_cv = cross_val_score(multi_output_regressor_knn,train_input,train_target,scoring='neg_mean_squared_error',cv=10)
    rmse_cv = np.sqrt(-scores_cv)
    print("\t ",red(f"KNN cv score : {rmse_cv}"))
    print("\t ",green(f"KNN cv RMSE average : {rmse_cv.mean():.2f}"))

    predictions = multi_output_regressor_knn.predict(test_input)

knn_regressor_predict()
def xgboost_regressor_predict():
    xg_reg = XGBRegressor()
    multi_output_regressor_knn = MultiOutputRegressor(xg_reg)
    multi_output_regressor_knn.fit(train_input, train_target)

    score = multi_output_regressor_knn.score(test_input, test_target)
    y_pred_xg = multi_output_regressor_knn.predict(test_input)
    mse = mean_squared_error(test_target, y_pred_xg)
    rmse = np.sqrt(mse)
    # print(yellow(f'KNN(3) regression model score: {score}'))
    print(yellow(f'XGBoost(3) regression model RMSE: {rmse:.2f}'))
    ### 교찯검증
    scores_cv = cross_val_score(multi_output_regressor_knn,train_input,train_target,scoring='neg_mean_squared_error',cv=10)
    rmse_cv = np.sqrt(-scores_cv)
    print("\t ",red(f"XGB cv score : {rmse_cv}"))
    print("\t ",green(f"XGB cv RMSE average : {rmse_cv.mean():.2f}"))
xgboost_regressor_predict()

Linear regression model RMSE: 21.18
	  LR cv score : [22.05600421 24.51390061 21.83705362 23.80306472 23.71399793 24.97807162
 22.20998485 22.53543424 25.11786173 26.24846072]
	  LR cv RMSE  average : 23.70
KNN(3) regression model RMSE: 17.47
	  KNN cv score : [18.10011084 19.95352427 18.58673623 20.51931024 19.53426041 19.86191078
 15.52137237 18.21313466 19.47105055 20.54933287]
	  KNN cv RMSE average : 19.03
XGBoost(3) regression model RMSE: 15.76
	  XGB cv score : [16.74510659 19.18668642 16.8309619  19.56155071 18.08791345 18.10144571
 15.25446328 17.02666954 17.84773678 19.01419169]
	  XGB cv RMSE average : 17.77


---
### XG boost modeling 

In [12]:
import platform; print(platform.platform())
import sys; print(sys.version)
import sklearn; print("Scikit-Learn", sklearn.__version__)
import xgboost; print("XGBoost", xgboost.__version__)

macOS-14.5-arm64-arm-64bit
3.11.7 (main, Dec 15 2023, 12:09:56) [Clang 14.0.6 ]
Scikit-Learn 1.2.2
XGBoost 2.0.3
